In [62]:
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

In [1]:
with open("../torsions/id_00073f8be.tor") as f:
    lines = list(f)

In [2]:
lines[21][88:]

' -68.2  BI\n'

In [58]:
def angle_to_flot(s):
    if '---' in s :
        return None
    else:
        return float(s)

In [57]:
def get_angles(lines):
    angles = []
    for line in lines[21:]:
        if line.startswith("***"):
            break
        rec = {}
        rec['chi'] = angle_to_flot(line[26:32])
        rec['alfa'] = angle_to_flot(line[40:46])
        rec['beta'] = angle_to_flot(line[48:54])
        rec['gamma'] = angle_to_flot(line[56:62])
        rec['delta'] = angle_to_flot(line[64:70])
        rec['epsilon'] = angle_to_flot(line[72:78])
        rec['zeta'] = angle_to_flot(line[80:86])
        rec['e-z'] = angle_to_flot(line[88:94])
        angles.append(rec)
    return angles

In [36]:
torsions = pd.DataFrame(get_angles()).values

In [39]:
np.cos(180)

7.446921003909191e+77

In [45]:
torsions_cos = np.cos(np.deg2rad(torsions))

In [46]:
torsions_sin = np.sin(np.deg2rad(torsions))

In [50]:
angles = np.concatenate([torsions_cos, torsions_sin],axis=-1)

In [55]:
angles[np.isnan(angles)] = -2

In [56]:
angles

array([[-0.97277588, -2.        , -0.9198215 , ..., -0.55629562,
        -0.9781476 , -0.92848583],
       [-0.97277588,  0.49545867, -0.9958844 , ..., -0.55193699,
        -2.        , -2.        ],
       [-0.99978068, -2.        , -2.        , ..., -0.9640954 ,
        -2.        , -2.        ],
       ...,
       [-0.97397597,  0.57357644, -0.9198215 , ...,  0.6481199 ,
        -2.        , -2.        ],
       [-0.9826128 , -2.        , -2.        , ..., -2.        ,
        -2.        , -2.        ],
       [ 0.99862953, -2.        , -2.        , ..., -2.        ,
        -2.        , -2.        ]])

In [64]:
def file_to_torsins(file):
    with file.open('r') as f:
        lines = list(f)
    torsions = pd.DataFrame(get_angles(lines)).values
    torsions_cos = np.cos(np.deg2rad(torsions))
    torsions_sin = np.sin(np.deg2rad(torsions))
    angles = np.concatenate([torsions_cos, torsions_sin],axis=-1)
    angles[np.isnan(angles)] = -2
    return angles

In [66]:
torsions = []
for file in tqdm(Path('../torsions/').glob("*.tor")):
    angles = file_to_torsins(file)
    torsions.append((file, angles))

In [69]:
!mkdir ../data/torsions

In [70]:
base = Path("../data/torsions/")

In [72]:
for file, tor in torsions:
    np.save(base/(file.stem+'.npy'),tor,allow_pickle=False)